# Natural Language Processing

## AllenNLP

AllenNLP is an open source library for building deep learning models for natural language processing, developed by the Allen Institute for Artificial Intelligence. It is built on top of PyTorch and is designed to support researchers, engineers, students, etc., who wish to build high quality deep NLP models with ease. It provides high-level abstractions and APIs for common components and models in modern NLP. It also provides an extensible framework that makes it easy to run and manage NLP experiments.

In a nutshell, AllenNLP is

- a library with well-thought-out abstractions encapsulating the common data and model operations that are done in NLP research
- a commandline tool for training PyTorch models
- a collection of pre-trained models that you can use to make predictions
- a collection of readable reference implementations of common / recent NLP models
- an experiment framework for doing replicable science
- a way to demo your research
- open source and community driven

In part 1, geared towards someone who is brand new to the library, we give you a quick walk-through of main AllenNLP concepts and features. We'll build a complete, working NLP model (a text classifier) along the way.

## Text Classification